In [ ]:
# csv 파일을 읽기 위해서 pandas를 import 함
import pandas as pd

# wine.csv 파일을 불러옴
f = pd.read_csv("/content/wine.csv")

# 첫번째 행(첫번째 데이터)의 모든 컬럼 정보를 불러옴
# iloc은 integer location으로 첫번째 인자는 첫번째 행, 두번째 인자는 전체 컬럼을 의미한다)
print(f.iloc[0,:])

In [ ]:
# f 데이터는 pandas Dataframe 데이터 타입이기 때문에 corr() 내장함수를 사용할 수 있다.
# 이는 각 컬럼끼리 페어하게 correlation coefficient 값을 출력해준다.
f.corr()

In [ ]:
# 바로 위 셀에서 f.corr()으로 alcohol과 상관계수가 플러스마이너스 0.25 이상인 컬럼들만 확인하고 
# f에서 volatile acidity, residual sugar, chlorides, total sulfur dioxide, density, quality, class 만 가져온다.
X = f[['volatile acidity','residual sugar','chlorides','total sulfur dioxide','density','quality','class']]

# target 값인 alcohol 컬럼만 가져온다.
y = f['alcohol']

# X는 즉 피쳐 데이터이고, 전체 데이터에서 모델 훈련에만 쓸 4000개의 데이터만 가져온다, 그리고 컬럼 또한 모두 가져온다.
X_train = X.iloc[:4000,:]
# X는 즉 피쳐 데이터이고, 전체 데이터에서 모델 validation에만 쓸 4000개이외의 데이터만 가져온다, 그리고 컬럼 또한 모두 가져온다.
X_valid = X.iloc[4000:,:]

# y는 훈련에 쓸 정답값 데이터다. 이것도 똑같이 X와 매칭해서 가져와야하니까 4000개만 슬라이싱해서 가져온다, 컬럼은 1개뿐이므로 따로 인덱싱하지 않아도 된다.
y_train = y.iloc[:4000]

# y는 validation에 쓸 정답값 데이터다. 
# 이것도 똑같이 X와 매칭해서 가져와야하니까 4000개이외만 슬라이싱해서 가져온다, 컬럼은 1개뿐이므로 따로 인덱싱하지 않아도 된다.
y_valid = y.iloc[4000:]

In [ ]:
# 데이터를 제대로 슬라이싱 했는지 갯수를 확인한다. 
print(len(X_train))
print(len(X_valid))
print(len(y_train))
print(len(y_valid))

In [ ]:
# 폴리노미얼 피쳐를 만들기 위해 sklearn의 PolynomialFeatures를 가지고옴
from sklearn.preprocessing import PolynomialFeatures

# PolynomialFeatures 함수를 통해 데이터(피처)를 n의 제곱까지의 피처를 늘릴 준비를 한다. 
# degree가 2 이고, 데이터의 컬럼이 A,B 이렇게라면 A^2, B^2, AB, A, B 이렇게 되겠다.
poly_features = PolynomialFeatures(degree=2, include_bias=False)

# X train이 가지고 있는 컬럼이 몇개인지 파악, 몇개까지의 feature를 늘려야하는지 분석함.(fit이라는 말에 속으면 안된다.)
poly_features.fit(X_train)

# X_train은 원래 7개 이지만 PolynomialFeatures 때문에 35개로 컬럼이 늘어난다. 이를 poly_X라는 새로운 X로 정의한다.
poly_X= poly_features.transform(X_train)

In [ ]:
# sklearn에서 지원해주는 SGDRegressor를 가져온다.
from sklearn.linear_model import SGDRegressor

# SGDRegressor 객체를 생성할건데 객체 안에 훈련 옵션을 셋팅한다. 경사하강법을 50번 반복할것이며(max_iter), loss를 언제까지 줄일건지(최대한 1e-3값 까지)
# 이외 옵션은 구글에서 검색해서 사람들이 대표적으로 사용하는 셋팅값을 불러와서 자주 쓴다.
sgd_reg = SGDRegressor(max_iter=50, tol=1e-3, penalty=None, eta0=0.1, random_state=42)

# PolynomialFeatures를 적용한 35개 컬럼의 poly_X와 그에 대응하는 정답값인 y_train를 모델한테 주고, SGD리그레서의 w 들을 학습시킨다.
sgd_reg.fit(poly_X, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDRegressor(eta0=0.1, max_iter=50, penalty=None, random_state=42)

In [ ]:
import numpy as np

# PolynomialFeatures는 이미 X_train의 컬럼이 몇개고, 몇개까지 늘려야하는지 분석했다, 
# 굳이 X_valid로 또 분석 시킬 필요가 없다(X_valid도 x_train이랑 피쳐 갯수가 똑같으니까)
# 그래서 그냥 위에서 PolynomialFeatures 객체를 poly_feature로 만든걸 그대로 가져와서 fit 시키지 않고 transform만 x_valid 한테 적용하면
# x_train과 똑같이 35개의 컬럼으로 늘어난다. 그 데이터를 X_new_poly라고 정의하는 것이다. 

X_new_poly = poly_features.transform(X_valid)
 

In [ ]:
# MSE를 가지고 예측값과 실제값의 차이를 측정할 것이다. 
from sklearn.metrics import mean_squared_error

# 훈련이 완료된 sgd_reg에다가 valid set이며, PolynomialFeatures를 적용시킨 35개 컬럼의 X_new_poly를 넣고 prediction 값을 뽑아낸다. 
y_predict = sgd_reg.predict(X_new_poly)

# MSE(예측값들, 실제 값들)
print(mean_squared_error(y_predict,y_valid))

8.025251371673693e+35
